In [18]:
import urllib.request, json 
import pandas as pd
from pandas.io.json import json_normalize
import requests


# resp = requests.get('https://www.metro.net/wp-json/wp/v2/line-override/')
# data = resp.json()
# pd.DataFrame(data)

resp = requests.get('https://www.metro.net/wp-json/wp/v2/line-override/', timeout=10,headers={'Content-Type': 'application/json'})
df= pd.read_json(resp.text)

cleaned_df = df.drop(['_links','_embedded','yoast_head','yoast_head_json'])

# cleaned_df.to_csv('resp.csv')


KeyError: "['_links' '_embedded' 'yoast_head' 'yoast_head_json'] not found in axis"

In [14]:

# to load the sample data we need pandas
df = pd.read_csv('test_data.csv')
sample_route_data = df.to_dict('records')
# sample_route_data_json = df.to_json(orient='index',index=['direction_id','stop_sequence'],indent=4)

sample_route_data_json = df.groupby('direction_id').apply(lambda x: x.to_json(orient='records'))


# d = {'departure_times':'departure_time'}

# works 2/6/2023
# j = df.groupby(['day_type']).apply(lambda x: x[['departure_times']].rename(columns=d).to_dict('records')).reset_index(name='depature_times').to_json('test_json3.json',orient='records',indent=4)
data_by_direction_id = df.groupby('direction_id').apply(lambda x: x.to_json(orient='records'))
group_direction_id_by_stop_id = df.groupby(['direction_id','stop_id']).apply(lambda x: x.to_json(orient='records'))

group_direction_id_by_stop_id_by_day_type = df.groupby(['direction_id','stop_id','day_type']).apply(lambda x: x.rename(columns=d).to_dict('records'))
group_direction_id_by_day_type_by_stop_sequence = df.groupby(['direction_id','day_type','stop_sequence']).apply(lambda x: x.rename(columns=d).to_dict('records'))

D = (group_direction_id_by_stop_id_by_day_type
  .groupby(level=0)
  .apply(lambda df: df.xs(df.name).to_dict())
  .to_dict()
)


# PANDAS IS VERY HELPFUL WHEN YOU ARE DOING:
#  - A JOIN (merging data from different tables)
#  - A GROUP BY (aggregating data by a certain column)
#  - TO ADD DATA TO A DATABASE QUICKLY


# 1. group data by direction_id
# 2. for each direction_id, group data by stop_id
# 3. for each stop_id, group data by day type <-- store this data as an object called "departure times"


# d = {'departure_times':'departure_time'}
# df.rename(columns=d, inplace=True)
# works as of 2.6.2023
d = (df.groupby(['direction_id','stop_id','day_type'])['departure_time','day_type']
       .apply(lambda x: x.to_dict('r'))
       .reset_index(name='data')
       .groupby(['direction_id','stop_id'])['day_type','data']
       .apply(lambda x: x.set_index('day_type')['data'].to_dict())
       .reset_index(name='departure_times')
       )
# group_direction_id_by_stop_id_by_day_type
group_direction_id_by_day_type_by_stop_sequence

FileNotFoundError: [Errno 2] No such file or directory: 'test_data.csv'

In [ ]:
# group_direction_id_by_stop_id_by_day_type.head(5)
df = pd.read_csv('test_data.csv')
def clean_up_group(x):
    #  x.set_index('day_type')['data'].to_dict()
     all_data = x.to_dict('records')
#      print(all_data)
     return all_data


# the function works!! stopped here 2/9/2023
# we need to get this to run on the entire dataframe in the larger script
def get_departure_times(target_data):
       initial_json = (target_data.groupby(['direction_id','day_type'])['departure_times','stop_sequence','stop_name','stop_id','coordinates']
       .apply(lambda x: x.to_dict('records'))
       .reset_index()
       .rename(columns={0:'data'})
       .groupby('direction_id')['day_type','data']
       .apply(lambda x:x.set_index('day_type')['data'].to_dict())
       .to_json(orient='index',indent=4)
       )
       cleaned_json = [json.loads(initial_json)]
       # initial_json.to_dict()
       return cleaned_json

final_result = pd.DataFrame()
final_result['route_code'] = df['route_code'].unique().astype('str')
final_result['route_id'] = df['route_id'].unique().astype('str')
final_result['payload'] = get_departure_times(df)

print(final_result.to_json('final_result.json',orient='records',indent=4))

test_json_10 = (df.groupby(['direction_id','day_type'])['departure_times','stop_sequence','stop_name','stop_id','coordinates']
       .apply(lambda x: x.to_dict('r'))
       .reset_index(name='data')
       .groupby('direction_id')['day_type','data']

       .apply(lambda x: x.set_index('day_type')['data'].to_dict())
       .to_json('test_json10.json',orient='index',indent=4)
       )
get_departure_times(df)
# d = (df.groupby(['direction_id','day_type','stop_sequence'])['departure_time','day_type']
#        .apply(lambda x: x.to_dict('r'))
#        .reset_index(name='data')
#        .groupby(['direction_id','day_type'])['stop_sequence','data']
#        .apply(lambda x: x.set_index('stop_sequence')['data'].to_dict())
#        .reset_index(name='departure_times')
#        .to_json('test_json10.json',orient='records',indent=4)
#        )


In [ ]:
import pandas as pd
import json
from sqlalchemy import create_engine

from sqlalchemy import create_engine,MetaData,event
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
import secrets as Config

# from .utils.log_helper import *

from secrets import *
engine = create_engine(Config.API_DB_URI, echo=False)

Session = sessionmaker(autocommit=False, autoflush=False, bind=engine)

session = Session()
target_schema = "metro_api_dev"
Base = declarative_base(metadata=MetaData(schema=target_schema))

def get_db():
    db = Session()
    try:
        print('Connected to the database')
        yield db
    finally:
        db.close()

# df = pd.read_sql_query()
# print('df.head(5)')
# print(df.head(5))
all_route_data= pd.read_csv('final_df.csv')
all_route_data.drop(columns=['id'],inplace=True)
all_route_data.sort_values(by=['route_code'],inplace=True)



def get_individual_route_data(target_data):
       target_data['day_type'] = target_data['day_type'].astype('str')
       initial_json = (target_data.groupby(['direction_id','day_type'])
       .apply(lambda x: x.to_dict('records'))
       .reset_index()
       .rename(columns={0:'data'})
       .groupby('direction_id')['day_type','data']
       .apply(lambda x:x.set_index('day_type')['data'].to_dict())
       .to_json(orient='index',indent=4)
       )
       cleaned_json = [json.loads(initial_json)]       
    #    cleaned_json = [json.loads(initial_json)]       
       # initial_json.to_dict()
       return cleaned_json

def get_departure_times(route):
    target_data = all_route_data.loc[all_route_data['route_code'].astype('str') == route]
    if target_data.shape[0] > 0:
        payload = []
        payload.append(get_individual_route_data(target_data))
        final_payload = pd.json_normalize(payload).assign(route_code=route).to_dict('records')
        return final_payload
    del target_data

final_df = pd.DataFrame()
route_code_df = pd.DataFrame({'route_code':all_route_data['route_code'].unique().astype('str')})
route_id_df = pd.DataFrame({'route_id':all_route_data['route_id'].unique().astype('str')})
final_df = pd.concat([route_code_df,route_id_df],ignore_index=True, axis=1)
final_df.columns = ['route_code','route_id']
# final_df['payload'] = final_df['route_code'].apply(lambda x: get_departure_times(x))
final_df['payload'] = final_df['route_code'].map(get_departure_times)

final_df['payload'] = final_df['payload'].astype('str')
final_df['payload'] = final_df['payload'].str.replace('0.weekday','weekday')
final_df['payload'] = final_df['payload'].str.replace('0.saturday','saturday')
final_df['payload'] = final_df['payload'].str.replace('0.sunday','sunday')
final_df['payload'] = final_df['payload'].str.replace('0.nan','none')
final_df['payload'] = final_df['payload'].str.replace('1.weekday','weekday')
final_df['payload'] = final_df['payload'].str.replace('1.saturday','saturday')
final_df['payload'] = final_df['payload'].str.replace('1.sunday','sunday')
final_df['payload'] = final_df['payload'].str.replace('1.nan','none')
final_df['payload'] = final_df['payload'].str.replace('1.weekday','weekday')
final_df['payload'] = final_df['payload'].str.replace('1.saturday','saturday')
final_df['payload'] = final_df['payload'].str.replace('1.sunday','sunday')
final_df['payload'] = final_df['payload'].str.replace('1.nan','none')
final_df['payload'] = final_df['payload'].str.replace('"','')
final_df.head().to_csv('final_evil_df.csv',index=False)
final_df.head().to_json('final_evil_df.json',orient='index',indent=4)
# final_df['payload'] = final_df['payload']


In [ ]:
final_df
final_df.head().to_csv('final_evil_df.csv',index=False)

In [ ]:
# print(group_direction_id_by_stop_id_by_day_type.to_frame())

# resource from stackoverflow: https://stackoverflow.com/questions/52923685/convert-pandas-multiindex-series-to-json-python
# method a
final_result = (df.groupby(['direction_id','stop_id'])['departure_time','day_type']
       .apply(lambda x: x.to_dict('r'))
       .reset_index(name='data')
       .groupby('direction_id')['stop_id','data']
       .apply(lambda x: x.set_index('stop_id')['data'].to_dict())
       .to_json('test_json8.json',orient='index',indent=4)
       )

# method b

d = (df.groupby(['direction_id','stop_id','day_type'])['departure_time','day_type']
       .apply(lambda x: x.to_dict('r'))
       .reset_index(name='data')
       .groupby(['direction_id','stop_id'])['day_type','data']
       .apply(lambda x: x.set_index('day_type')['data'].to_dict())
       .reset_index(name='departure_times')
       # .apply(lambda x: x.set_index('day_type')['data'].to_dict())
       # .groupby('stop_id')['direction_id','data']
       # .apply(lambda x: x.set_index('direction_id')['data'].to_dict())
       .to_json('test_json6.json',orient='index',indent=4)
       )


print(d)
# final_result

In [ ]:
# new code
d = {'departure_times':'departure_time'}

def process_data(x):
    # x.drop('day_type', axis=1, inplace=True)  
    # print(x.day_type)
    result = x.set_index('day_type')['data'].to_dict()
    # result.to_json('test_reulst.json',orient='index',indent=4)
    return result

df.rename(columns=d, inplace=True)
d = (df.groupby(['direction_id','stop_id','day_type'])['departure_time','day_type']
       .apply(lambda x: x.to_dict('r'))
       .reset_index(name='data')
       .groupby(['direction_id','stop_id'])['day_type','data']
       .apply(lambda x: process_data(x))
       .reset_index(name='departure_times')
       # .apply(lambda x: x.set_index('day_type')['data'].to_dict())
       # .groupby('stop_id')['direction_id','data']
       # .apply(lambda x: x.set_index('direction_id')['data'].to_dict())
    #    .to_json('test_json8.json',orient='index',indent=4)
       )

e = d.groupby(['direction_id'])['stop_id','departure_times'].apply(lambda x: x.set_index('stop_id')['departure_times'].to_dict()).reset_index(name='stop_id')
e.to_json('test_json6.json',orient='records',indent=4)


In [ ]:
d = {'departure_times':'departure_time'}
df.rename(columns=d, inplace=True)
# dataframe_by_direction_id = df.groupby('direction_id').apply(lambda x: x.to_dict('r')).to_frame()
# dataframe_by_direction_id = dataframe_by_direction_id.rename(columns={0:'route_id'}).apply(lambda x: x.to_dict('r')).to_frame()
# print(dataframe_by_direction_id)
# from collections import defaultdict

d = (df.groupby(['direction_id','stop_id','day_type'])['departure_time','day_type']
       .apply(lambda x: x.to_dict('r'))
       .reset_index(name='data')
       .groupby(['direction_id','stop_id'])['day_type','data']
       .apply(lambda x: x.set_index('day_type')['data'].to_dict())
       .reset_index(name='departure_times')
       # .apply(lambda x: x.set_index('day_type')['data'].to_dict())
       # .groupby('stop_id')['direction_id','data']
       # .apply(lambda x: x.set_index('direction_id')['data'].to_dict())
       # .to_frame(orient='index')
       # .to_json('test_json6.json',orient='index',indent=4)
       )

# print(d)
d


In [ ]:
# group_direction_id_by_stop_id_by_day_type = df.groupby(['direction_id','stop_id','day_type']).apply(lambda x: x.rename(columns=d).to_dict('records'))


# need to get the direction_id to be the key at the top level
# need to get the stop_id to be the key at the second level - done
# need to get the day_type to be the key at the third level - done
# need to get the departure_time to be the value at the fourth level - done



d = (df.groupby(['direction_id','stop_id','day_type'])['departure_time','day_type']
       .apply(lambda x: x.to_dict('r'))
       .reset_index(name='data')
       .groupby(['direction_id','stop_id'])['day_type','data']
       .apply(lambda x: x.set_index('day_type')['data'].to_dict())
       .reset_index(name='departure_times')
       # .apply(lambda x: x.set_index('day_type')['data'].to_dict())
       # .groupby('stop_id')['direction_id','data']
       # .apply(lambda x: x.set_index('direction_id')['data'].to_dict())
       # .to_json('test_json6.json',orient='index',indent=4)
       )


### Goal `json` v2

``` js
{
    route_id: '',
    '0': {
        weekday: [
            {
                stop_sequence: 1,
                stop_id: '',
                stop_name: '',
                coordinate: '',
                departure_times: ['', '', '']
            },{
                stop_sequence: 2,
                stop_id: '',
                stop_name: '',
                coordinate: '',
                departure_times: ['', '', '']
            },{
                stop_sequence: 3,
                stop_id: '',
                stop_name: '',
                coordinate: '',
                departure_times: ['', '', '']
            }
        ],
        saturday: [

        ],
        sunday: [

        ]
    },
    '1': {
        
    }
```

### The goal `json`

``` js
{
  route_id: '',
  direction_0:
  [
    {
      stop_id: '',
      stop_name: '',
      stop_sequence: '',
      coordinate: '',
      departure_times: 
      {
        weekday:
        [
          {
            departure_time: ''
          },{
            departure_time: ''
          },{
            departure_time: ''
          },{
            departure_time: ''
          }
        ],
        saturday:
        [
          {
            trip_id: '',
            departure_time: ''
          },{
            trip_id: '',
            departure_time: ''
          },{
            trip_id: '',
            departure_time: ''
          },{
            trip_id: '',
            departure_time: ''
          }
        ],
        sunday:
        [
          {
            trip_id: '',
            departure_time: ''
          },{
            trip_id: '',
            departure_time: ''
          },{
            trip_id: '',
            departure_time: ''
          },{
            trip_id: '',
            departure_time: ''
          }
        ]
      }
    },{
      stop_id: '',
      stop_name: '',
      stop_sequence: '',
      coordinate: '',
      departure_times: 
      {
        weekday:
        [
          {
            trip_id: '',
            departure_time: ''
          },{
            trip_id: '',
            departure_time: ''
          },{
            trip_id: '',
            departure_time: ''
          },{
            trip_id: '',
            departure_time: ''
          }
        ],
        saturday:
        [
          {
            trip_id: '',
            departure_time: ''
          },{
            trip_id: '',
            departure_time: ''
          },{
            trip_id: '',
            departure_time: ''
          },{
            trip_id: '',
            departure_time: ''
          }
        ],
        sunday:
        [
          {
            trip_id: '',
            departure_time: ''
          },{
            trip_id: '',
            departure_time: ''
          },{
            trip_id: '',
            departure_time: ''
          },{
            trip_id: '',
            departure_time: ''
          }
        ]
      }
    }
  ],
  direction_1: 
  [
    
  ]
}

```